## Number Plate Detecttion

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as pt
import matplotlib.pyplot as plt
import cv2 as cv2
import skimage.io as io
import xml.etree.ElementTree as xet
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.callbacks import TensorBoard


In [2]:
train_df = pd.read_csv("/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train.csv")

In [3]:
train_df.head(10)

,img_id,ymin,xmin,ymax,xmax
0,1.jpg,276,94,326,169
1,10.jpg,311,395,344,444
2,100.jpg,406,263,450,434
3,101.jpg,283,363,315,494
4,102.jpg,139,42,280,222
5,103.jpg,437,320,475,450
6,104.jpg,482,511,547,623
7,105.jpg,281,452,330,535
8,106.jpg,309,172,332,281
9,107.jpg,399,328,425,441


### Creating a CSV file from the "test" folder containing 201 images

In [32]:

image_folder = "/kaggle/input/number-plate-detection-recognition/test/test/test"  # Update this path

# Get all image filenames (JPG, PNG, etc.)
image_paths = glob(os.path.join(image_folder, "*.jpg"))  # Change extension if needed

# Extract only the filenames (not full paths)
image_names = [os.path.basename(img) for img in image_paths]

# Create a DataFrame
testdf = pd.DataFrame({'img_id': image_names})

# Save as CSV
csv_path = "images.csv"
testdf.to_csv(csv_path, index=False)

print(f"CSV saved successfully: {csv_path}")
print(testdf.head(10))  

CSV saved successfully: images.csv
     img_id
0  1075.jpg
1  1031.jpg
2  1024.jpg
3  1009.jpg
4   915.jpg


In [34]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img_id  210 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


#### adding the filepath to the actual images for train_df and testdf to later input the scaled images in form of arrays

In [10]:
filename = train_df['img_id'][0]
def getFilename(filename):
    filepath_image = os.path.join('/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/', filename)
    return filepath_image
getFilename(filename)

'/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/1.jpg'

In [35]:
filename2 = testdf['img_id'][0]
def getFilename(filename2):
    filepath_image2 = os.path.join('/kaggle/input/number-plate-detection-recognition/test/test/test/', filename2)
    return filepath_image2
getFilename(filename2)

'/kaggle/input/number-plate-detection-recognition/test/test/test/1075.jpg'

In [11]:
image_path = list(train_df['img_id'].apply(getFilename))
image_path[:15]

['/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/1.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/10.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/100.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/101.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/102.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/103.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/104.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetection_train/license_plates_detection_train/105.jpg',
 '/kaggle/input/number-plate-detection-recognition/Licplatesdetecti

In [36]:
image_path2 = list(testdf['img_id'].apply(getFilename))
image_path2[:15]

['/kaggle/input/number-plate-detection-recognition/test/test/test/1075.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1031.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1024.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1009.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/915.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1055.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1026.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/930.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1010.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1025.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1058.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/1101.jpg',
 '/kaggle/input/number-plate-detection-recognition/test/test/test/988.jpg',
 '

#### just checking how an image with its xmin, xmax, ymin and ymax show up

In [13]:
file_path = image_path[2] #path of our image N2.jpeg
img = cv2.imread(file_path) #read the image
img = io.imread(file_path) #Read the image
fig = pt.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 1 - 100.jpg with bounding box')
fig.add_shape(type='rect',x0=434, x1=263, y0=450, y1=406, xref='x', yref='y',line_color='cyan')

#### back to converting the actual images into arrays after scaling them to the standard for the ML model

In [37]:
labels = train_df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [38]:
data2 = []
for ind in range(len(image_path2)):
    image2 = image_path2[ind]
    img_arr2 = cv2.imread(image2)
    h2,w2,d2 = img_arr2.shape
    # Prepprocesing
    load_image2 = load_img(image2,target_size=(224,224))
    load_image_arr2 = img_to_array(load_image2)
    norm_load_image_arr2 = load_image_arr2/255.0 # Normalization
    # Append
    data2.append(norm_load_image_arr2)

## Initiating the ML model instance InceptionResNetV2
Inception-ResNet-v2 is a convolutional neural network architecture that combines the Inception family of architectures with residual connections, replacing the filter concatenation stage of the Inception architecture. It's a deeper version of the Inception V3 model and is trained on a large dataset like ImageNet. 
Here's a more detailed breakdown:

**Architecture:**
* Inception-ResNet-v2 builds upon the Inception architecture, known for its use of multiple convolutional filters of different sizes to capture features at various scales. 
* It incorporates residual connections, which allow information to flow directly from earlier layers to later layers, addressing the vanishing gradient problem and enabling the training of deeper networks. 
* It replaces the filter concatenation stage of the Inception architecture with residual connections.
   
**Training:**
* Inception-ResNet-v2 is trained on a large dataset like ImageNet, which contains millions of labeled images.
* This training process allows the network to learn to classify images into 1000 object categories.
  
**Usage:**
* Inception-ResNet-v2 can be used for image classification, object detection, and other computer vision tasks. 
* It's a popular choice for transfer learning, where the pre-trained network is used as a starting point for fine-tuning on a new dataset. 



In [15]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [16]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 111, 111, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 111, 111, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 109, 109, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 109, 109, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 109, 109, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 109, 109, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 109, 109, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 54, 54, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 54, 54, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 54, 54, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 52, 52, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 52, 52, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 73,602,946 (280.77 MB)

 Non-trainable params: 60,544 (236.50 KB)

In [18]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [39]:
X_test = np.array(data2,dtype=np.float32)

In [40]:
X_test.shape

(210, 224, 224, 3)

### Training the train_df after separating it into X for image array form and y for label (xmin, xmax, ymin and ymax) array form

In [19]:
tfb = TensorBoard('object_detection')
history = model.fit(x=X,y=y,batch_size=20,epochs=100,callbacks=[tfb])

Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 164s 328ms/step - loss: 0.0623
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 330ms/step - loss: 0.0160
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 335ms/step - loss: 0.0084
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 336ms/step - loss: 0.0061
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 341ms/step - loss: 0.0048
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 345ms/step - loss: 0.0046
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 349ms/step - loss: 0.0048
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 353ms/step - loss: 0.0039
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 356ms/step - loss: 0.0037
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 355ms/step - loss: 0.0040
Epoch 11/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 360ms/step - loss: 0.0040
Epoch 12/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 361ms/step - loss: 0.0039
Epoch 13/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 364ms/step - loss: 0.0039
Epoch 14/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 16s 366ms/step - loss: 0.0036
Epoch 15/100
45/45 ━━━━━━━━━

### using our trained model on the testdf and checking the first five predictions

In [41]:
predictions = model.predict(X_test)

print(predictions[:5]) 

7/7 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step 
[[0.8856063  0.3261676  0.74122125 0.51491773]
 [0.61441183 0.3206645  0.8368411  0.80888456]
 [0.43001667 0.62545234 0.637705   0.95228153]
 [0.41930997 0.37989274 0.7637169  0.9912116 ]
 [0.446908   0.22901992 0.84395695 0.69326305]]


In [42]:
def denormalize_bbox(norm_bbox, img_width, img_height):
    nxmin, nxmax, nymin, nymax = norm_bbox
    xmin = int(nxmin * img_width)
    xmax = int(nxmax * img_width)
    ymin = int(nymin * img_height)
    ymax = int(nymax * img_height)
    return xmin, xmax, ymin, ymax

In [43]:
for i, pred in enumerate(predictions[:5]):  # Checking first 5 predictions
    xmin, xmax, ymin, ymax = denormalize_bbox(pred, 224, 224)
    print(f"Image {i+1}: Bounding Box - xmin: {xmin}, xmax: {xmax}, ymin: {ymin}, ymax: {ymax}")


Image 1: Bounding Box - xmin: 198, xmax: 73, ymin: 166, ymax: 115
Image 2: Bounding Box - xmin: 137, xmax: 71, ymin: 187, ymax: 181
Image 3: Bounding Box - xmin: 96, xmax: 140, ymin: 142, ymax: 213
Image 4: Bounding Box - xmin: 93, xmax: 85, ymin: 171, ymax: 222
Image 5: Bounding Box - xmin: 100, xmax: 51, ymin: 189, ymax: 155
